In [35]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [37]:
from openai import OpenAI
import json
import time
from typing import List, Dict, Any, Tuple, Union

# from core.features.content_development.python.prompts import SIMPLE
from core.features.topic_segregation.utils import add_dicts, calculate_cost_gpt4_8k, calculate_cost_gpt4_turbo
from core.features.provider import creator, text_model_defaults

from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

In [5]:
"""
FLOW - 
1. generate a course structure after asking for a topic, scope of the topic, duration of course (in hours), parts (modules)
format ```json 

{
    "topic": "topic",
    "curriculum": {
        "module1" : [list of subtopics in module1],
        "module2" : [list of subtopics in module2],
            ...
    },
}

2. take first topic of module1, and ask for the content on it with proper comments and explanations. add all the response in different files and iterate for all the topics of all the modules. 


pseudo code - 


structure = generate_structure(topic, topic_scope, duration_in_hours, number_of_parts)

for module in structure["curriculum"]:
    for topic in structure["curriculum"][module]:
        content = generate_content(topic)
        save_content(content)


"""

'\nFLOW - \n1. generate a course structure after asking for a topic, scope of the topic, duration of course (in hours), parts (modules)\nformat ```json \n\n{\n    "topic": "topic",\n    "curriculum": {\n        "module1" : [list of subtopics in module1],\n        "module2" : [list of subtopics in module2],\n            ...\n    },\n}\n\n2. take first topic of module1, and ask for the content on it with proper comments and explanations. add all the response in different files and iterate for all the topics of all the modules. \n\n\npseudo code - \n\n\nstructure = generate_structure(topic, topic_scope, duration_in_hours, number_of_parts)\n\nfor module in structure["curriculum"]:\n    for topic in structure["curriculum"][module]:\n        content = generate_content(topic)\n        save_content(content)\n\n\n'

In [ ]:
conversation = [{"role": "system", "content": "You are a code generation engine. For every topic give by the user, you will provide a comprehensive tutorial jupyter notebook. You iwll output all responses in raw JSON format as it would appear in a Jupyter Notebook. Include all relevant fields such as cell type, source code, execution count, and outputs. Ensure the output is properly structured and formatted as per Jupyter Notebook's JSON specifications."}]

user_message = {'role': 'user', 'content': """write a jupyter notebook's raw json to create a comprehensive tutorial for python dictionaries. 

in the notebook include, just include cell_type and source.
Your output should be a json only."""}

conversation.append(user_message)


response = creator(**text_model_defaults, messages=conversation)

output = response.choices[0].message.content

print(output)

In [39]:
raw_notebook = json.loads(output)

In [45]:
import json

def convert_to_jupyter_notebook(input_json, file_name):

    # Template for a Jupyter Notebook

    """
    Rules to create the jupyter notebook:
    1. metadata tag must be present in code cell
    """
    notebook_template = {
        "cells": [],
        "metadata": {'kernelspec': {'display_name': 'Python 3',
            'language': 'python',
            'name': 'python3'},
            'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
            'file_extension': '.py',
            'mimetype': 'text/x-python',
            'name': 'python',
            'nbconvert_exporter': 'python',
            'pygments_lexer': 'ipython3',
            'version': '3.8.5'}},
        "nbformat": 4,
        "nbformat_minor": 4
    }

    # Adding the cells from the input JSON to the notebook template
    for cell in input_json['cells']:
        if cell['cell_type'] == "code":
            cell['metadata'] = {}

    notebook_template['cells'] = input_json['cells']

    with open(f'{file_name}.json', 'w') as file:
        json.dump(notebook_template, file, indent=2)


# Convert to Jupyter Notebook JSON
jupyter_notebook_json = convert_to_jupyter_notebook(raw_notebook, file_name='python_dict_new')

